In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

In [ ]:
from contracts.builtin_contracts import SFControlContract

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)

Start

Control epochs one by one

In [ ]:
from time import sleep
epocs_to_advance = 5
for i in range(epocs_to_advance):
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    context.admin_account.sync_nonce(context.network_provider.proxy)
    sf_control_contract.epochs_fast_forward(context.admin_account, context.network_provider.proxy, 1, 5)
    sleep(60)
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')

Control epochs in batch

In [ ]:
from time import sleep
epocs_to_advance = 22
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')
context.admin_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

In [ ]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Transfer eGLD

In [ ]:
from utils.utils_chain import Account, WrapperAddress, nominated_amount
from utils.utils_tx import endpoint_call

sender = Account(pem_file=config.DEFAULT_ADMIN)
receiver = WrapperAddress("")
amount = nominated_amount(1)

sender.sync_nonce(context.network_provider.proxy)
endpoint_call(context.network_provider.proxy, 100000, sender, receiver, "cado", [], amount)

Transfer ESDTs

In [ ]:
from utils.utils_chain import Account, WrapperAddress
from utils.utils_tx import ESDTToken, multi_esdt_transfer

sender = Account(pem_file=config.DEFAULT_ADMIN)

sender.address = WrapperAddress('')
receiver = WrapperAddress('')

tokens = [
    ESDTToken('TSLA-4C9', 0, 1000000000000000000),
]

sender.sync_nonce(context.network_provider.proxy)
multi_esdt_transfer(context.network_provider.proxy, 1000000, sender, receiver, tokens)

XMEX Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk_network_providers import ApiNetworkProvider

searched_offset = 30

energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token

# get current epoch
main_api = ApiNetworkProvider("https://api.multiversx.com")
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')

for i in range(1, 10000):
    result = main_api.get_non_fungible_token(locked_token, i)
    if result:
        decoded_data = decode_merged_attributes(base64_to_hex(result.attributes), XMEX_ATTRIBUTES)
        print(i, decoded_data)
        if current_epoch < decoded_data['unlock_epoch'] < current_epoch + searched_offset:
            print(i)
        if decoded_data['unlock_epoch'] < current_epoch:
            print(i)
            break
    

XMEXFARM Token searcher

In [ ]:
from utils.decoding_structures import XMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes
from utils.utils_chain import base64_to_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
from utils.decoding_structures import XMEXFARM_ATTRIBUTES, XMEXLP_ATTRIBUTES, XMEX_ATTRIBUTES, LKMEX_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, dec_to_padded_hex
from multiversx_sdk_network_providers import ApiNetworkProvider
# Get token details for a given farm token

searched_offset = 360*3

energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token = energy_contract.locked_token
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]

# get current epoch
main_api = ApiNetworkProvider('https://api.multiversx.com')
used_api = main_api
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')

current_nonce = int('0472a0', 16)

for i in range(1, 10000):
    token_nonce = current_nonce - i
    print(f"Checking token {token_nonce} {dec_to_padded_hex(token_nonce)}")
    
    try:
        farm_token_on_network = used_api.get_non_fungible_token(proxy_contract.proxy_farm_token, token_nonce)
    except Exception as e:
        print(f"Failed to get token {proxy_contract.proxy_farm_token} with nonce {token_nonce}")
        continue

    # Decode the farm token attributes
    decoded_xmex_farm_attributes = decode_merged_attributes(base64_to_hex(farm_token_on_network.attributes), XMEXFARM_ATTRIBUTES)
    # print(decoded_xmex_farm_attributes)

    # Decode the LP token attributes
    xmex_lp_token_id = decoded_xmex_farm_attributes.get('proxy_token_id')
    if xmex_lp_token_id != proxy_contract.proxy_lp_token:
        print(f"FAIL!!! Wrong token contained by XMEXFARM token: {xmex_lp_token_id} expected {proxy_contract.proxy_lp_token}")

    try:
        lp_token_on_network = used_api.get_non_fungible_token(xmex_lp_token_id, decoded_xmex_farm_attributes.get('proxy_token_nonce'))
    except Exception as e:
        print(f"Failed to get token {xmex_lp_token_id} with nonce {decoded_xmex_farm_attributes.get('proxy_token_nonce')}")
        continue

    decoded_xmex_lp_attributes = decode_merged_attributes(base64_to_hex(lp_token_on_network.attributes), XMEXLP_ATTRIBUTES)
    # print(decoded_xmex_lp_attributes)

    # Decode the XMEX token attributes
    xmex_token_id = decoded_xmex_lp_attributes.get('locked_tokens_id')

    xmex_token_on_network = used_api.get_non_fungible_token(xmex_token_id, decoded_xmex_lp_attributes.get('locked_tokens_nonce'))

    if "XMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), XMEX_ATTRIBUTES)
    if "LKMEX" in xmex_token_id:
        decoded_attributes = decode_merged_attributes(base64_to_hex(xmex_token_on_network.attributes), LKMEX_ATTRIBUTES)
        print(decoded_attributes)

    # if decoded_attributes.get('unlock_epoch', 10000000000) < current_epoch + searched_offset:
    #     print(f"Found token {token_nonce} : {dec_to_padded_hex(token_nonce)} with unlock epoch: {decoded_attributes.get('unlock_epoch')}")
    #     continue
    